## 1.5 이종 그래프 (Heterogeneous Graph
- 다른 타입의 노드와 에지를 갖는 그래프
- 다른 타입의 노드/에지는 독립적인 ID 공간과 피처 저장소를 가짐
- 아래 그램의 예를 보면, user와 game 노드 ID는 모두 0부터 시작하고, 서로 다른 피처들을 갖고 있ㅇ므
<img src = "https://data.dgl.ai/asset/image/user_guide_graphch_2.png">\
두 타입의 노드(user와 game)와 두 타입의 에지(follows와 plays)를 갖는 이종 그래프 예

#### 1. 이종 그래프 생성하기
- DGL에서 이종 그래프(짧게 heterograph)는 관계당 하나의 그래프들의 시리즈로 표현
- 각 관계는 문자열 트리플 `(source node type, edge type, destination node type)` 
- 관계가 에지 타입을 명확하게 하기 때문에, DGL은 이것들을 캐노니컬(canonical) 에지 타입이라고 함

In [11]:
# DGL에서 이종 그래프를 만드는 예제
import dgl
import torch as th
# Create a heterograph with 3 node types and 3 edges types.
graph_data = {
   ('drug', 'interacts', 'drug'): (th.tensor([0, 1]), th.tensor([1, 2])),
   ('drug', 'interacts', 'gene'): (th.tensor([0, 1]), th.tensor([2, 3])),
   ('drug', 'treats', 'disease'): (th.tensor([1]), th.tensor([2]))
}
g = dgl.heterograph(graph_data)
g.ntypes
g.etypes
g.canonical_etypes

[('drug', 'interacts', 'drug'),
 ('drug', 'interacts', 'gene'),
 ('drug', 'treats', 'disease')]

In [12]:
# 특별한 이종 그래프의 예 : 동종(homogeneous) 및 이분(bipartite) 그래프 = 하나의 관계를 갖는 그래프
uh, vh = th.tensor([0, 1]), th.tensor([1, 2])
ub, vb = th.tensor([0, 1]), th.tensor([2, 3])
# A homogeneous graph
dgl.heterograph({('node_type', 'edge_type', 'node_type'): (uh, vh)})
# A bipartite graph
dgl.heterograph({('source_type', 'edge_type', 'destination_type'): (ub, vb)})

Graph(num_nodes={'destination_type': 4, 'source_type': 2},
      num_edges={('source_type', 'edge_type', 'destination_type'): 2},
      metagraph=[('source_type', 'destination_type', 'edge_type')])

- 이종 그래프와 연관된 메타그래프(metagraph) 는 그래프의 스키마
- 이것은 노드들과 노드간의 에지들의 집합에 대한 타입 제약 조건을 지정
- 메타그래프의 노드 u 는 연관된 이종 그래프의 노드 타입에 해당
- 메타그래프의 에지 (u,v) 는 연관된 이종 그래프의 노드 타입 u 와 노드 타입 v 간에 에지가 있다는 것을 알려줌

In [13]:
g
g.metagraph().edges()

OutMultiEdgeDataView([('drug', 'drug'), ('drug', 'gene'), ('drug', 'disease')])

- 참고할 API들: `dgl.heterograph() , ntypes , etypes , canonical_etypes , metagraph`

#### 2. 다양한 타입을 다루기
- 노드와 에지가 여러 타입이 사용되는 경우, 타입 관련된 정보를 위한 DGL Graph API를 호출할 때는 노드/에지의 타입을 명시해야 함
- 추가로 다른 타입의 노드/에지는 별도의 ID를 가짐

In [14]:
# Get the number of all nodes in the graph
g.num_nodes()
# Get the number of drug nodes
g.num_nodes('drug')
# Nodes of different types have separate IDs,
# hence not well-defined without a type specified
#g.nodes()
g.nodes('drug')

tensor([0, 1, 2])

In [15]:
# 특정 노드/에지 타입에 대한 피쳐를 설정하고 얻을 때, DGL에서 제공하는 두가지 새로운 형태의 문법
# g.nodes[‘node_type’].data[‘feat_name’]`,  `g.edges[‘edge_type’].data[‘feat_name’]

# Set/get feature 'hv' for nodes of type 'drug'
g.nodes['drug'].data['hv'] = th.ones(3, 1)
g.nodes['drug'].data['hv']
# Set/get feature 'he' for edge of type 'treats'
g.edges['treats'].data['he'] = th.zeros(1, 1)
g.edges['treats'].data['he']

tensor([[0.]])

In [16]:
# 만약 그래프가 오직 한개의 노드/에지 타입을 갖는다면, 노드/에지 타입을 명시할 필요가 없다
g = dgl.heterograph({
   ('drug', 'interacts', 'drug'): (th.tensor([0, 1]), th.tensor([1, 2])),
   ('drug', 'is similar', 'drug'): (th.tensor([0, 1]), th.tensor([2, 3]))
})
g.nodes()
# To set/get feature with a single type, no need to use the new syntax
g.ndata['hv'] = th.ones(4, 1)

- Note
    - 에지 타입이 목적지와 도착지 노드의 타입을 고유하게 결정할 수 있다면, 에지 타입을 명시할 때 문자 트리플 대신 한 문자만들 사용 가능
    - 예 : 두 관계 `('user', 'plays', 'game')` 과 `('user', 'likes', 'game')`를 갖는 이종 그래프가 있을 때, 두 관계를 지정하기 위해서 단지 `'plays'` 또는 `'likes'` 를 사용해도 됨

#### 3. 디스크에서 이종 그래프 로딩하기
##### (1) Comma Separated Values (CSV) : 이종 그래프를 저장하는 일반적인 방법은 다른 타입의 노드와 에지를 서로 다른 CSV 파일에 저장하는 것
``````
# data folder
data/
|-- drug.csv        # drug nodes
|-- gene.csv        # gene nodes
|-- disease.csv     # disease nodes
|-- drug-interact-drug.csv  # drug-drug interaction edges
|-- drug-interact-gene.csv  # drug-gene interaction edges
|-- drug-treat-disease.csv  # drug-treat-disease edges
``````
##### (2) DGL 바이너리 포멧
- DGL은 이종 그래프를 바이너리 포멧으로 저장하고 읽기 위한 함수 `dgl.save_graphs()` 와 `dgl.load_graphs()` 를 제공

#### 4. 에지 타입 서브그래프
- 보존하고 싶은 관계를 명시
- 피처가 있을 경우는 이를 복사하면서 이종 그래프의 서브그래프를 생성

In [17]:
g = dgl.heterograph({
   ('drug', 'interacts', 'drug'): (th.tensor([0, 1]), th.tensor([1, 2])),
   ('drug', 'interacts', 'gene'): (th.tensor([0, 1]), th.tensor([2, 3])),
   ('drug', 'treats', 'disease'): (th.tensor([1]), th.tensor([2]))
})
g.nodes['drug'].data['hv'] = th.ones(3, 1)
# Retain relations ('drug', 'interacts', 'drug') and ('drug', 'treats', 'disease')
# All nodes for 'drug' and 'disease' will be retained
eg = dgl.edge_type_subgraph(g, [('drug', 'interacts', 'drug'),
                                ('drug', 'treats', 'disease')])
eg
# The associated features will be copied as well
eg.nodes['drug'].data['hv']

tensor([[1.],
        [1.],
        [1.]])

#### 5. 이종 그래프를 동종 그래프로 변환하기
- 이종 그래프는 다른 타입의 노드/에지와 그것들에 연관된 피쳐들을 관리하는데 깔끔한 인터페이스를 제공
- 아래의 경우 특히 유용
    1. 다른 타입의 노드/에지에 대한 피쳐가 다른 데이터 타입 또는 크기를 갖는다.
    2. 다른 타입의 노드/에지에 다른 연산을 적용하고 싶다.
- 만약 위 조건을 만족하지 않고 모델링에서 노드/에지 타입의 구별이 필요하지 않는다면 : DGL의 `dgl.DGLGraph.to_homogeneous()` API를 이용해서 이종 그래프를 동종 그래프로 변환 가능 (아래의 절차를 따름)
    1. 모든 타입의 노드/에지를 0부터 시작하는 정수로 레이블을 다시 부여한다.
    2. 사용자가 지정한 노드/에지 타입들에 걸쳐서 피쳐들을 합친다.

In [18]:
g = dgl.heterograph({
   ('drug', 'interacts', 'drug'): (th.tensor([0, 1]), th.tensor([1, 2])),
   ('drug', 'treats', 'disease'): (th.tensor([1]), th.tensor([2]))})
g.nodes['drug'].data['hv'] = th.zeros(3, 1)
g.nodes['disease'].data['hv'] = th.ones(3, 1)
g.edges['interacts'].data['he'] = th.zeros(2, 1)
g.edges['treats'].data['he'] = th.zeros(1, 2)
# By default, it does not merge any features
hg = dgl.to_homogeneous(g)
'hv' in hg.ndata
# Copy edge features
# For feature copy, it expects features to have
# the same size and dtype across node/edge types
hg = dgl.to_homogeneous(g, edata=['he'])
# Copy node features
hg = dgl.to_homogeneous(g, ndata=['hv'])
hg.ndata['hv']

DGLError: Cannot concatenate column he with shape Scheme(shape=(2,), dtype=torch.float32) and shape Scheme(shape=(1,), dtype=torch.float32)

원래의 노드/에지 타입과 타입별 ID들은 ndata 와 edata 에 저장

In [19]:
# Order of node types in the heterograph
g.ntypes
# Original node types
hg.ndata[dgl.NTYPE]
# Original type-specific node IDs
hg.ndata[dgl.NID]
# Order of edge types in the heterograph
g.etypes
# Original edge types
hg.edata[dgl.ETYPE]
# Original type-specific edge IDs
hg.edata[dgl.EID]

tensor([0, 1, 0])

##### 모델링 목적으로, 특정 관계들을 모아서 그룹으로 만들고, 그것들에 같은 연산을 적용하고 싶은 경우
1. 이종 그래프의 에지 타입 서브그래프를 추출
2. 그 서브그래프를 동종 그래프로 변환

In [20]:
g = dgl.heterograph({
   ('drug', 'interacts', 'drug'): (th.tensor([0, 1]), th.tensor([1, 2])),
   ('drug', 'interacts', 'gene'): (th.tensor([0, 1]), th.tensor([2, 3])),
   ('drug', 'treats', 'disease'): (th.tensor([1]), th.tensor([2]))
})
sub_g = dgl.edge_type_subgraph(g, [('drug', 'interacts', 'drug'),
                                   ('drug', 'interacts', 'gene')])
h_sub_g = dgl.to_homogeneous(sub_g)
h_sub_g

Graph(num_nodes=7, num_edges=4,
      ndata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64), '_TYPE': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64), '_TYPE': Scheme(shape=(), dtype=torch.int64)})

## 1.6 GPU에서 DGLGraph 사용하기
그래프 생성시, 두 GPU 텐서를 전달해서 GPU에 위치한 `DGLGraph` 를 만들 수 있다. 다른 방법으로는 `to()` API를 사용해서 `DGLGraph` 를 GPU로 복사할 수 있다. 이는 그래프 구조와 피처 데이터를 함께 복사한다.

In [1]:
import dgl
import torch as th
u, v = th.tensor([0, 1, 2]), th.tensor([2, 3, 4])
g = dgl.graph((u, v))
g.ndata['x'] = th.randn(5, 3)  # original feature is on CPU
g.device
cuda_g = g.to('cuda:0')  # accepts any device objects from backend framework
cuda_g.device
cuda_g.ndata['x'].device       # feature data is copied to GPU too
# A graph constructed from GPU tensors is also on GPU
u, v = u.to('cuda:0'), v.to('cuda:0')
g = dgl.graph((u, v))
g.device

c:\Users\yang.yoonjeong\anaconda3\envs\torch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DGLError: [14:17:30] C:\Users\Administrator\dgl-0.5\src\runtime\c_runtime_api.cc:88: Check failed: allow_missing: Device API gpu is not enabled. Please install the cuda version of dgl.

``````
For any future person coming here with the same problem (uninstalling DLG and still finding the package to be available) just print its path:

python3 -c 'import dgl; print(dgl.__path__)'

and remove it.
``````

GPU 그래프에 대한 모든 연산은 GPU에서 수행된다. 따라서, 모든 텐서 인자들이 GPU에 이미 존재해야하며, 연산 결과(그래프 또는 텐서) 역시 GPU에 저장된다. 더 나아가, GPU 그래프는 GPU에 있는 피쳐 데이터만 받아들인다.

In [2]:
cuda_g.in_degrees()
cuda_g.in_edges([2, 3, 4])   # ok for non-tensor type arguments
cuda_g.in_edges(th.tensor([2, 3, 4]).to('cuda:0'))  # tensor type must be on GPU
cuda_g.ndata['h'] = th.randn(5, 4)  # ERROR! feature must be on GPU too!

NameError: name 'cuda_g' is not defined